In [ ]:
%run ./check_DOVS_METHODS.ipynb

In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import copy

import itertools

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

In [ ]:
# date_0 = '2023-01-01'
# date_1 = '2023-01-31'
# states=['OH']
# opcos='oh'

outg_rec_nbs = [
    '13330651', 
    '13325577', 
    '13329763', 
    '13278397', 
    '13183482', 
    '13183481'
]

conn_outages = Utilities.get_utldb01p_oracle_connection()

In [ ]:
sql_outage_full = DOVSOutages_SQL.build_sql_std_outage(
    mjr_mnr_cause=None, 
    include_premise=True, 
    outg_rec_nbs=outg_rec_nbs, 
#     date_range=[date_0, date_1], 
#     states=states, 
    MJR_CAUSE_CD=None, 
    DEVICE_CD=None, 
    INTRPTN_TYP_CD=None, 
    CURR_REC_STAT_CD=None
).get_sql_statement()
#-----
print(f'sql_outage_full:\n{sql_outage_full}\n\n')
#-----
df_outage_OG = pd.read_sql_query(
    sql_outage_full, 
    conn_outages, 
    dtype={
        'CI_NB':np.int32, 
        'CMI_NB':np.float64, 
        'OUTG_REC_NB':np.int32
    }
)
#-----
print(f"df_outage_OG.shape = {df_outage_OG.shape}")
print(f"# OUTG_REC_NBs     = {df_outage_OG['OUTG_REC_NB'].nunique()}")

In [ ]:
outg_rec_nbs_all = df_outage_OG['OUTG_REC_NB'].unique().tolist()
df_outage = df_outage_OG.copy()

In [ ]:
df_outage.drop(columns=['OFF_TM', 'REST_TM', 'PREMISE_NB']).drop_duplicates()

In [ ]:
df_outage.columns.tolist()

In [ ]:
df_outage.drop(columns=['OFF_TM', 'REST_TM', 'PREMISE_NB']).drop_duplicates()[['OUTG_REC_NB', 'STATE_ABBR_TX']]

In [ ]:
outg_rec_nbs_all

In [ ]:
start=time.time()
#-----
df_mp_outg_OG = build_active_MP_for_xfmrs_in_outages_df(
    df_outage=df_outage, 
    prem_nb_col='PREMISE_NB', 
    is_slim=False, 
    addtnl_mp_df_curr_cols=['technology_tx', 'state_cd'], 
    addtnl_mp_df_hist_cols=['technology_tx', 'state_cd']
)
#-----
print(f'Time for build_active_MP_for_xfmrs_in_outages_df: {time.time()-start}')
start=time.time()
#-----
df_mp_outg_OG['inst_ts'] = pd.to_datetime(df_mp_outg_OG['inst_ts'])
df_mp_outg_OG['rmvl_ts'] = pd.to_datetime(df_mp_outg_OG['rmvl_ts'])
#-------------------------
df_mp_outg = MeterPremise.drop_approx_mp_duplicates(
    mp_df = df_mp_outg_OG.copy(), 
    fuzziness=pd.Timedelta('1 hour'), 
    assert_single_overlap=True, 
    addtnl_groupby_cols=['OUTG_REC_NB', 'technology_tx', 'state_cd'], 
    gpby_dropna=False
)
#-----
print(f'Time for drop_approx_mp_duplicates: {time.time()-start}')

In [ ]:
# Some premises are listed in DOVS are simply not found in AMI
print(f"#PNs DOVS:            {df_outage['PREMISE_NB'].nunique()}")
print(f"#PNs AMI using xfmrs: {df_mp_outg['prem_nb'].nunique()}")

In [ ]:
set(df_mp_outg['prem_nb'].unique().tolist()).difference(set(df_outage['PREMISE_NB'].unique().tolist()))

In [ ]:
set(df_outage['PREMISE_NB'].unique().tolist()).difference(set(df_mp_outg['prem_nb'].unique().tolist()))

In [ ]:
# Really only want one entry per meter (here, meter being a mfr_devc_ser_nbr/prem_nb combination)
# ALthough drop_duplicates was used, multiple entries could still exist if, e.g., a meter has two
#   non-fuzzy-overlapping intervals
assert(all(df_mp_outg[['mfr_devc_ser_nbr', 'prem_nb', 'OUTG_REC_NB']].value_counts()==1))

# # Simple-minded (if assertion fails): Let's just keep the one with the most recent install date
# df_mp_outg = df_mp_outg.iloc[df_mp_outg.reset_index().groupby(['mfr_devc_ser_nbr', 'prem_nb', 'OUTG_REC_NB'])['inst_ts'].idxmax()]
# assert(all(df_mp_outg[['mfr_devc_ser_nbr', 'prem_nb', 'OUTG_REC_NB']].value_counts()==1))

In [ ]:
df_mp_outg.groupby(['OUTG_REC_NB']).apply(lambda x: 100*(x[x['technology_tx']=='AMI'].shape[0]/x.shape[0]))

In [ ]:
df_mp_outg

In [ ]:
check_df = DOVSOutages.merge_df_outage_with_mp(
    df_outage=df_outage.copy(), 
    df_mp=df_mp_outg.copy(), 
    merge_on_outg=['OUTG_REC_NB', 'PREMISE_NB'], 
    merge_on_mp=['OUTG_REC_NB', 'prem_nb'], 
    cols_to_include_mp=None, 
    drop_cols = None, 
    rename_cols=None, 
    inplace=True
)

In [ ]:
check_df.columns.tolist()

In [ ]:
check_df[['STATE_ABBR_TX', 'state_cd']]

In [ ]:
check_df['STATE_ABBR_TX'].equals(check_df['state_cd'])

In [ ]:
check_df['STATE_ABBR_TX']==check_df['state_cd']

In [ ]:
check_df[check_df['STATE_ABBR_TX']!=check_df['state_cd']][['STATE_ABBR_TX', 'state_cd']]

In [ ]:
mico_outage_nbs = [
    1455461, 
    1431681, 
    1475011, 
    2322101, 
    2188221, 
    2188221
]

mico_outg_rec_nbs = [
    13330651, 
    13325577, 
    13329763, 
    13278397, 
    13183482, 
    13183481
]

In [ ]:
mico_outage_outg_map = dict(zip(mico_outg_rec_nbs, mico_outage_nbs))

In [ ]:
df_mp_outg['OUTAGE_NB'] = df_mp_outg['OUTG_REC_NB'].map(mico_outage_outg_map)

In [ ]:
df1 = df_mp_outg.groupby(['OUTG_REC_NB', 'OUTAGE_NB']).apply(lambda x: 100*(x[x['technology_tx']=='AMI'].shape[0]/x.shape[0])).to_frame(name='%AMI')
df1

In [ ]:
# Some premises are listed in DOVS are simply not found in AMI
print(f"#PNs DOVS:            {df_outage['PREMISE_NB'].nunique()}")
print(f"#PNs AMI using xfmrs: {df_mp_outg['prem_nb'].nunique()}")

In [ ]:
df_mp_outg.groupby(['OUTG_REC_NB', 'OUTAGE_NB']).apply(lambda x: print(x.name[0]))

In [ ]:
df2 = df_mp_outg.groupby(['OUTG_REC_NB', 'OUTAGE_NB']).apply(
    lambda x: len(set(df_outage[df_outage['OUTG_REC_NB']==x.name[0]]['PREMISE_NB'].unique().tolist()).difference(set(x['prem_nb'].unique().tolist())))
).to_frame(name='# PNs missing')
df2

In [ ]:
df3 = df_mp_outg.groupby(['OUTG_REC_NB', 'OUTAGE_NB']).apply(
    lambda x: df_outage[df_outage['OUTG_REC_NB']==x.name[0]]['PREMISE_NB'].nunique()
).to_frame(name='# PNs total')
df3

In [ ]:
df12 = pd.merge(df1, df2, how='inner', left_index=True, right_index=True)
df12

In [ ]:
df123 = pd.merge(df12, df3, how='inner', left_index=True, right_index=True)
df123

In [ ]:
# ami_outg_rec_nbs

In [ ]:
len(set(df_outage['PREMISE_NB'].unique().tolist()).difference(set(df_mp_outg['prem_nb'].unique().tolist())))

In [ ]:
df_outage[df_outage['OUTG_REC_NB']==13183481]

In [ ]:
# # Keep only trsf_pole_nbs with all meters of type AMI
# df_mp_outg_ami = df_mp_outg.groupby(['trsf_pole_nb']).filter(lambda x: all(x['technology_tx']=='AMI'))

# Keep on outages with all meters of typr AMI
df_mp_outg_ami = df_mp_outg.groupby(['OUTG_REC_NB']).filter(lambda x: all(x['technology_tx']=='AMI'))

In [ ]:
print(df_mp_outg.shape)
print(df_mp_outg_ami.shape)

In [ ]:
ami_outg_rec_nbs

In [ ]:
ami_outg_rec_nbs = df_mp_outg_ami['OUTG_REC_NB'].unique().tolist()
df_outage_ami = df_outage[df_outage['OUTG_REC_NB'].isin(ami_outg_rec_nbs)]

In [ ]:
df_outage_ami_w_mp = DOVSOutages.merge_df_outage_with_mp(
    df_outage=df_outage_ami.copy(), 
    df_mp=df_mp_outg_ami, 
    merge_on_outg=['OUTG_REC_NB', 'PREMISE_NB'], 
    merge_on_mp=['OUTG_REC_NB', 'prem_nb'], 
    cols_to_include_mp=None, 
    drop_cols = None, 
    rename_cols=None, 
    inplace=True
)

In [ ]:
df_outage_w_mp = DOVSOutages.merge_df_outage_with_mp(
    df_outage=df_outage.copy(), 
    df_mp=df_mp_outg, 
    merge_on_outg=['OUTG_REC_NB', 'PREMISE_NB'], 
    merge_on_mp=['OUTG_REC_NB', 'prem_nb'], 
    cols_to_include_mp=None, 
    drop_cols = None, 
    rename_cols=None, 
    inplace=True
)

In [ ]:
df_outage_ami_w_mp=df_outage_w_mp

In [ ]:
df_outage_ami_w_mp['OUTG_REC_NB'].nunique()

In [ ]:
df_outage_ami_w_mp

In [ ]:
df_outage_ami_w_mp_slim = DOVSOutages.consolidate_df_outage(df_outage_ami_w_mp)

In [ ]:
df_outage_ami_w_mp_slim

In [ ]:
df_outage_ami_w_mp_slim = DOVSOutages.set_search_time_in_outage_df(
    df_outage=df_outage_ami_w_mp_slim, 
    search_time_half_window=pd.Timedelta('24 hours')
)

In [ ]:
df_outage_ami_w_mp_slim

In [ ]:
df_outage_ami_w_mp_slim['OUTG_REC_NB'].nunique()

## AMI NonVee

In [ ]:
#-------------------------
usg_split_to_CTEs=True
df_construct_type=DFConstructType.kRunSqlQuery
contstruct_df_args_ami=None
addtnl_groupby_cols=['OUTG_REC_NB', 'trsf_pole_nb']

cols_of_interest_ami = TableInfos.AMINonVee_TI.std_columns_of_interest
batch_size=10
verbose=True
n_update=1

In [ ]:
# ami_sql_function_kwargs = dict(
#     cols_of_interest=cols_of_interest_ami, 
#     df_outage=df_outage_ami_w_mp_slim, 
#     split_to_CTEs=usg_split_to_CTEs, 
#     join_mp_args=False, 
#     df_args = dict(
#         addtnl_groupby_cols=addtnl_groupby_cols, 
#         mapping_to_ami={'PREMISE_NBS':'premise_nbs'}, 
#         is_df_consolidated=True
#     ), 
#     field_to_split='df_outage', 
#     field_to_split_location_in_kwargs=['df_outage'], 
# #     save_and_dump=True, 
#     save_and_dump=False, 
#     sort_coll_to_split=True,
#     batch_size=batch_size, verbose=verbose, n_update=n_update
# )
# # addtnl_ami_sql_function_kwargs = dict(
# #     build_sql_function_kwargs=dict(opco=opcos)
# # )
# # ami_sql_function_kwargs = {**ami_sql_function_kwargs, 
# #                            **addtnl_ami_sql_function_kwargs}

In [ ]:
# start=time.time()
# ami_nonvee = AMINonVee(
#     df_construct_type=df_construct_type, 
#     contstruct_df_args = contstruct_df_args_ami, 
#     build_sql_function=AMINonVee_SQL.build_sql_usg_for_outages, 
#     build_sql_function_kwargs=ami_sql_function_kwargs, 
#     init_df_in_constructor=True, 
#     save_args=False
# )
# build_time = time.time()-start
# print(build_time)

In [ ]:
ami_sql_function_kwargs = dict(
    cols_of_interest=cols_of_interest_ami, 
    df_outage=df_outage_ami_w_mp_slim, 
    split_to_CTEs=usg_split_to_CTEs, 
    join_mp_args=False, 
    df_args = dict(
        addtnl_groupby_cols=addtnl_groupby_cols, 
        mapping_to_ami={'PREMISE_NBS':'premise_nbs'}, 
        is_df_consolidated=True
    ), 
    field_to_split='df_outage', 
    field_to_split_location_in_kwargs=['df_outage'], 
    save_and_dump=True,  
    sort_coll_to_split=True,
    batch_size=batch_size, verbose=verbose, n_update=n_update
)
# addtnl_ami_sql_function_kwargs = dict(
#     build_sql_function_kwargs=dict(opco=opcos)
# )
# ami_sql_function_kwargs = {**ami_sql_function_kwargs, 
#                            **addtnl_ami_sql_function_kwargs}


save_args = dict(
    save_to_file=True, 
    save_dir = r'C:\Users\s346557\Documents\LocalData\dovs_check\forMico\AMINonVee', 
    save_name=r'ami_nonvee.csv', 
    index=True
)

In [ ]:
start=time.time()
ami_nonvee = AMINonVee(
    df_construct_type=df_construct_type, 
    contstruct_df_args = contstruct_df_args_ami, 
    build_sql_function=AMINonVee_SQL.build_sql_usg_for_outages, 
    build_sql_function_kwargs=ami_sql_function_kwargs, 
    init_df_in_constructor=True, 
    save_args=save_args
)
build_time = time.time()-start
print(build_time)

# AMI End Events

In [ ]:
#-------------------------
usg_split_to_CTEs=True
df_construct_type=DFConstructType.kRunSqlQuery
contstruct_df_args_end_events=None
addtnl_groupby_cols=['OUTG_REC_NB', 'trsf_pole_nb']

cols_of_interest_end_dev_event = TableInfos.AMIEndEvents_TI.std_columns_of_interest
batch_size=10
verbose=True
n_update=1

In [ ]:
end_events_sql_function_kwargs = dict(
    cols_of_interest=cols_of_interest_end_dev_event, 
    df_outage=df_outage_ami_w_mp_slim, 
    split_to_CTEs=usg_split_to_CTEs, 
    join_mp_args=False, 
    df_args = dict(
        addtnl_groupby_cols=addtnl_groupby_cols, 
        mapping_to_ami={'PREMISE_NBS':'premise_nbs'}, 
        is_df_consolidated=True
    ), 
    field_to_split='df_outage', 
    field_to_split_location_in_kwargs=['df_outage'], 
    save_and_dump=True, 
    sort_coll_to_split=True,
    batch_size=batch_size, verbose=verbose, n_update=n_update
)
# addtnl_end_events_sql_function_kwargs = dict(
#     build_sql_function_kwargs=dict(opco=opcos)
# )
# end_events_sql_function_kwargs = {**end_events_sql_function_kwargs, 
#                                   **addtnl_end_events_sql_function_kwargs}

end_events_save_args = dict(
    save_to_file=True, 
    save_dir = r'C:\Users\s346557\Documents\LocalData\dovs_check\forMico\EndEvents', 
    save_name=r'end_events.csv', 
    index=True
)

In [ ]:
start=time.time()
end_events = AMIEndEvents(
    df_construct_type=df_construct_type, 
    contstruct_df_args = contstruct_df_args_end_events, 
    build_sql_function=AMIEndEvents_SQL.build_sql_end_events_for_outages, 
    build_sql_function_kwargs=end_events_sql_function_kwargs, 
    init_df_in_constructor=True, 
    save_args=end_events_save_args
)
end_events_build_time = time.time()-start